In [1]:
import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리
from selenium import webdriver   # 웹 브라우저 자동화
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller

import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
from tqdm import tqdm_notebook   # for문 돌릴 때 진행상황을 %게이지로 알려준다.
from pandas.io.html import read_html
# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# Step 1. 크롬 웹브라우저 실행
# pip install chromedriver_autoinstaller
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path)

# 사이트 주소는 네이버
driver.get('https://1xbet.whoscored.com/')
time.sleep(2)  # 2초간 정지
# 'statistics' 클릭
driver.find_element_by_link_text("Statistics").click( )
time.sleep(1)
driver.execute_script("window.scrollTo(0, 1200)")
time.sleep(1)

#click '공격수' button
time.sleep(2)
offense = driver.find_element_by_id('top-player-stats-options').find_element_by_link_text('Offensive')
offense.click()

#click '모든 버튼' button
time.sleep(2)
all_player = driver.find_element_by_link_text('All players')
all_player.click()

page = driver.find_element_by_id('statistics-paging-offensive').find_element_by_link_text('last')
total_page = int(page.get_attribute('data-page'))

print(total_page)
df_offensive = pd.DataFrame(columns = ['Player','Apps', 'Mins', 'Goals', 'Assists', 'SpG'\
       ,'KeyP', 'Drb', 'Fouled', 'Off','Disp','UnsTch','Rating'])


table = driver.find_element_by_xpath('//*[@id="statistics-table-offensive"]')
# 다음 페이지 클릭

for i in np.arange(total_page/14):
    time.sleep(5)
    
    table_html= table.get_attribute('innerHTML')
    df2 = read_html(table_html)[0]
    
    df_offensive = pd.concat([df_offensive, df2], ignore_index=True )
    driver.find_element_by_id('statistics-paging-offensive').find_element_by_link_text('next').click()

151


In [2]:
df_offensive

,Player,Apps,Mins,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating,Player.1
0,"1Mattia ZaccagniVerona, 26, AM(CL),FW",2,180,2,-,2,2.5,3.5,3.5,-,4,1,8.47,"Mattia Zaccagni Verona, 26, AM(CL),FW"
1,"2Robert LewandowskiBayern, 33, FW",18,1541,20,1,4.6,1.4,1.5,1,0.4,1.3,2.3,8.10,"Robert Lewandowski Bayern, 33, FW"
2,"3Malang SarrChelsea, 22, D(CL)",1,90,-,-,-,-,-,-,-,-,-,7.95,"Malang Sarr Chelsea, 22, D(CL)"
3,"4Erling HaalandBorussia Dortmund, 21, FW",11(1),992,13,5,4.3,1,0.8,0.7,0.5,0.6,2.8,7.88,"Erling Haaland Borussia Dortmund, 21, FW"
4,"5Mohamed SalahLiverpool, 29, AM(CLR),FW",20,1784,16,9,4,2.1,1.9,0.8,0.5,2.1,3.3,7.82,"Mohamed Salah Liverpool, 29, AM(CLR),FW"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,"106Zlatan IbrahimovicAC Milan, 40, FW",9(4),790,8,2,3,1.2,0.2,0.5,1.1,0.5,1.2,7.17,"Zlatan Ibrahimovic AC Milan, 40, FW"
106,"107André-Frank Zambo AnguissaNapoli, 26, DMC",13(1),1178,-,-,1.1,1,2.1,1.4,-,1.1,1.7,7.16,"André-Frank Zambo Anguissa Napoli, 26, DMC"
107,"108Ruslan MalinovskyiAtalanta, 28, M(CR),FW",13(5),1092,5,2,2.3,1.9,1.8,1.4,0.1,1.9,2.4,7.16,"Ruslan Malinovskyi Atalanta, 28, M(CR),FW"
108,"109Reece JamesChelsea, 22, D(CR),DMC,M(R)",14(2),1118,4,5,1.3,1.9,1.1,1.1,0.2,0.9,1,7.16,"Reece James Chelsea, 22, D(CR),DMC,M(R)"


In [3]:
# data preprocessing
df = df_offensive.drop(['Player'],axis = 1).reset_index()
df = df.drop(['index'],axis = 1)
df

,Apps,Mins,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating,Player.1
0,2,180,2,-,2,2.5,3.5,3.5,-,4,1,8.47,"Mattia Zaccagni Verona, 26, AM(CL),FW"
1,18,1541,20,1,4.6,1.4,1.5,1,0.4,1.3,2.3,8.10,"Robert Lewandowski Bayern, 33, FW"
2,1,90,-,-,-,-,-,-,-,-,-,7.95,"Malang Sarr Chelsea, 22, D(CL)"
3,11(1),992,13,5,4.3,1,0.8,0.7,0.5,0.6,2.8,7.88,"Erling Haaland Borussia Dortmund, 21, FW"
4,20,1784,16,9,4,2.1,1.9,0.8,0.5,2.1,3.3,7.82,"Mohamed Salah Liverpool, 29, AM(CLR),FW"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,9(4),790,8,2,3,1.2,0.2,0.5,1.1,0.5,1.2,7.17,"Zlatan Ibrahimovic AC Milan, 40, FW"
106,13(1),1178,-,-,1.1,1,2.1,1.4,-,1.1,1.7,7.16,"André-Frank Zambo Anguissa Napoli, 26, DMC"
107,13(5),1092,5,2,2.3,1.9,1.8,1.4,0.1,1.9,2.4,7.16,"Ruslan Malinovskyi Atalanta, 28, M(CR),FW"
108,14(2),1118,4,5,1.3,1.9,1.1,1.1,0.2,0.9,1,7.16,"Reece James Chelsea, 22, D(CR),DMC,M(R)"


In [4]:
name = [] 
age = [] 
pos1 = []
pos2 = []

spl = df['Player.1'].str.split(',')
for i in range(len(spl)):
    a = spl[i][0]
    name.append(a)
df['name'] = name

for i in range(len(spl)):
    a = int(spl[i][1])
    age.append(a)
df['age'] = age

for i in range(len(spl)):
    a = spl[i][2]
    pos1.append(a)
df['pos1']=pos1

for i in range(len(spl)):
    if len(spl[i]) > 3 :
        a = spl[i][3]
    else : a = np.nan
    pos2.append(a)
df['pos2'] = pos2

  

In [5]:
df = df.iloc[:,[13,14,15,16,0,1,2,3,4,5,6,7,8,9,10,11,12]]
df.to_csv("result.csv", encoding='utf-8-sig')


In [6]:
df

,name,age,pos1,pos2,Apps,Mins,Goals,Assists,SpG,KeyP,Drb,Fouled,Off,Disp,UnsTch,Rating,Player.1
0,Mattia Zaccagni Verona,26,AM(CL),FW,2,180,2,-,2,2.5,3.5,3.5,-,4,1,8.47,"Mattia Zaccagni Verona, 26, AM(CL),FW"
1,Robert Lewandowski Bayern,33,FW,NaN,18,1541,20,1,4.6,1.4,1.5,1,0.4,1.3,2.3,8.10,"Robert Lewandowski Bayern, 33, FW"
2,Malang Sarr Chelsea,22,D(CL),NaN,1,90,-,-,-,-,-,-,-,-,-,7.95,"Malang Sarr Chelsea, 22, D(CL)"
3,Erling Haaland Borussia Dortmund,21,FW,NaN,11(1),992,13,5,4.3,1,0.8,0.7,0.5,0.6,2.8,7.88,"Erling Haaland Borussia Dortmund, 21, FW"
4,Mohamed Salah Liverpool,29,AM(CLR),FW,20,1784,16,9,4,2.1,1.9,0.8,0.5,2.1,3.3,7.82,"Mohamed Salah Liverpool, 29, AM(CLR),FW"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Zlatan Ibrahimovic AC Milan,40,FW,NaN,9(4),790,8,2,3,1.2,0.2,0.5,1.1,0.5,1.2,7.17,"Zlatan Ibrahimovic AC Milan, 40, FW"
106,André-Frank Zambo Anguissa Napoli,26,DMC,NaN,13(1),1178,-,-,1.1,1,2.1,1.4,-,1.1,1.7,7.16,"André-Frank Zambo Anguissa Napoli, 26, DMC"
107,Ruslan Malinovskyi Atalanta,28,M(CR),FW,13(5),1092,5,2,2.3,1.9,1.8,1.4,0.1,1.9,2.4,7.16,"Ruslan Malinovskyi Atalanta, 28, M(CR),FW"
108,Reece James Chelsea,22,D(CR),DMC,14(2),1118,4,5,1.3,1.9,1.1,1.1,0.2,0.9,1,7.16,"Reece James Chelsea, 22, D(CR),DMC,M(R)"
